In [ ]:
!pip install langchain
!pip install pypdf
!pip install langchain-groq
!pip install chromadb #vector database

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

pdf1 = "/content/deep_work.pdf"

pdf2 = "https://arxiv.org/pdf/2005.11401.pdf"

loaders = [
  # Duplicate documents on purpose - messy data
  PyPDFLoader(pdf1),
  PyPDFLoader(pdf1),
  PyPDFLoader(pdf2),
]

docs = []
for i, loader in enumerate(loaders):
  pages = loader.load()
  print(f"For doc = {i}, number of pages: {len(pages)}")
  docs.extend(loader.load())

print(f"Length of docs {len(docs)}")

For doc = 0, number of pages: 8
For doc = 1, number of pages: 8
For doc = 2, number of pages: 19
 length of docs 35


In [ ]:
# Chunking documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 1500,
  chunk_overlap = 150,
  separators=['\n\n','. ']
)

chunks = text_splitter.split_documents(docs)

len(chunks)

80

In [32]:
from google.colab import userdata
from langchain.vectorstores import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=userdata.get('hf_api'), model_name="sentence-transformers/all-MiniLM-l6-v2"  ##need to use this model due to size constraint in
    #hugging face serverless inference api
)

In [33]:
persist_directory = './chroma'

vectordb = Chroma.from_documents(
  documents=chunks,
  embedding=embedding,
  persist_directory=persist_directory
)

In [35]:
question = "What is a deep work"

docs_similarity_search = vectordb.similarity_search(question, k=3)

for doc in docs_similarity_search:
  print(doc.page_content[:200], f"==> metadata = {doc.metadata}")

. This kind of work
means we don't create anything of value. So why is it that we gravitate
towards shallow work?
The truth is that shallow work is easy, and deep work is difficult.
Furthermore, shall ==> metadata = {'page': 1, 'source': '/content/deep_work.pdf'}
. This kind of work
means we don't create anything of value. So why is it that we gravitate
towards shallow work?
The truth is that shallow work is easy, and deep work is difficult.
Furthermore, shall ==> metadata = {'page': 1, 'source': '/content/deep_work.pdf'}
All of the best, and most creative work, emerges from a state of clear
focus and careful attention. So, perhaps deep work, along with restorative
rest is just the antidote we need. Deep Work is a guid ==> metadata = {'page': 7, 'source': '/content/deep_work.pdf'}


In [36]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key=userdata.get('groq_api'),
    temperature=0,
    model = 'Llama3-8b-8192'
)

In [37]:
# retrieval qa chains
# 1- Base chain: Include the whole context in the query to the LLM

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=vectordb.as_retriever(),
  return_source_documents=True,
)

In [38]:
question = "What is a deep work"
result = qa_chain({"query": question})
print(f"Answer:\n {result['result']}")

Answer:
 According to the provided text, deep work refers to a type of work that requires clear focus and careful attention. It is described as difficult to do because it requires uninterrupted time and dedication to one's craft. Deep work is contrasted with shallow work, which is easy and deceptive, disguising itself as productivity but adding little value to one's work output.


In [39]:
#get source documents
for doc in result['source_documents']:
  print(doc.page_content[:200], f"==> metadata = {doc.metadata}\n")

. This kind of work
means we don't create anything of value. So why is it that we gravitate
towards shallow work?
The truth is that shallow work is easy, and deep work is difficult.
Furthermore, shall ==> metadata = {'page': 1, 'source': '/content/deep_work.pdf'}

. This kind of work
means we don't create anything of value. So why is it that we gravitate
towards shallow work?
The truth is that shallow work is easy, and deep work is difficult.
Furthermore, shall ==> metadata = {'page': 1, 'source': '/content/deep_work.pdf'}

All of the best, and most creative work, emerges from a state of clear
focus and careful attention. So, perhaps deep work, along with restorative
rest is just the antidote we need. Deep Work is a guid ==> metadata = {'page': 7, 'source': '/content/deep_work.pdf'}

All of the best, and most creative work, emerges from a state of clear
focus and careful attention. So, perhaps deep work, along with restorative
rest is just the antidote we need. Deep Work is a guid ==> 

In [40]:
# MMR retriever
qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=vectordb.as_retriever(search_type = "mmr"),
  return_source_documents=True,
)

question = "What is a deep work"
result = qa_chain({"query": question})
print(f"Answer:\n {result['result']}")

Answer:
 According to the provided text, deep work refers to a state of clear focus and careful attention, where one dedicates uninterrupted time to their craft. It is described as difficult to achieve because it requires spending time without distractions, which is hard to do. Deep work is contrasted with shallow work, which is easy to do but adds little value to one's work output.


In [42]:
# 2-Map-reduce chain: Each individual chunk is sent to the LLM, to get a base answer. Then those answers are composed
# to get the final answer

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=vectordb.as_retriever(),
  chain_type="map_reduce"
)

question = "What is a deep work?"
result = qa_chain({"query": question})
print(f"Answer:\n {result['result']}")

Answer:
 According to the provided text, "Deep work" is difficult because it's hard to spend uninterrupted time routinely dedicated to your craft.


In [44]:
# 3-refine chain: each chunk is passed to llm composed with the answer from the previous llm

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=vectordb.as_retriever(),
  chain_type="refine"
)

question = "What is a deep work"
result = qa_chain({"query": question})
print(f"Answer:\n {result['result']}")

Answer:
 With the additional context, I can refine the original answer to provide a more accurate definition of deep work.

Deep work refers to a state of clear focus and careful attention, where one can eliminate distractions and improve overall focus. It is a guide that shows us that we can take back control of our time, eliminate distractions, and improve our overall focus. In essence, deep work is the antidote to the constant distractions and noise that can hinder productivity and creativity.


In [45]:
# 4 - Map rerank: Each chunk is passed to the LLM and the answer from the each chunk is ranked. Chunk with
# higher rank is the result

qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=vectordb.as_retriever(),
  chain_type="map_rerank"
)
question = "What is a deep work?"
result = qa_chain({"query": question})
print(f"Answer:\n {result['result']}")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Answer:
 Helpful Answer: Deep work is difficult because it's hard to spend uninterrupted time routinely dedicated to your craft.


In [46]:
# using custom prompt template
from langchain.prompts import PromptTemplate
template = """Use the provided context to respond to the question posed at the end. If you're unsure of the answer, please feel free to acknowledge that you don't know rather than attempting to provide a fabricated response.
Please provide a brief and concise response.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], template="Use the provided context to respond to the question posed at the end. If you're unsure of the answer, please feel free to acknowledge that you don't know rather than attempting to provide a fabricated response.\nPlease provide a brief and concise response.\n{context}\nQuestion: {question}\nHelpful Answer:")

In [47]:
qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=vectordb.as_retriever(),
  return_source_documents=True,
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "What is a deep work"
result = qa_chain({"query": question})
print(f"Answer:\n {result['result']}")

Answer:
 According to the provided context, a deep work is a type of work that requires uninterrupted time and focus, allowing individuals to dedicate themselves to their craft. It is described as difficult to achieve due to the ease of shallow work and the distractions that come with it.
